In [2]:
import random as rd
import simpy

## Simulação Inicial

In [3]:
#variáveis da simulação
SEED = True
RANDOM_SEED = 616
NUM_ATENDENTES = 4
SIM_TIME = 200

In [4]:
class CallCentre:

    def __init__(self,env,num_atendentes):
        self.env = env
        self.atendente = simpy.Resource(env,num_atendentes)
        self.espera = []


    def atendimento(self): # executa o tempo de atendimento

        tempo_de_atendimento = 150 # definir e variar valor

        yield self.env.timeout(tempo_de_atendimento)
        #print(f"Cliente {cliente} é atendido as {self.env.now}")

def cliente(env,nome,CC): # gera um cliente

    inicio = env.now

    print(f"Cliente {nome} solicita atendimento as {inicio}")

    with CC.atendente.request() as request:

        yield request
        atendimento = env.now
        print(f"Cliente {nome} inicia atendimento as {atendimento}")

        yield env.process(CC.atendimento(nome))
        print(f"Cliente {nome} desliga as {env.now}")
        CC.espera.append(atendimento)


def run(env,num_atendentes): # executa a simulação
    nome = 1

    call_centre = CallCentre(env,num_atendentes)

    env.process(cliente(env,nome,call_centre))

    while True:
        yield env.timeout(rd.randint(2,10)) # definir e variar valor
        nome += 1
        env.process(cliente(env,nome,call_centre))

    return call_centre.espera

In [5]:
if SEED: rd.seed(RANDOM_SEED)

env = simpy.Environment()
env.process(run(env,NUM_ATENDENTES))

print("saída:")
env.run(until=SIM_TIME)

saída:
Cliente 1 solicita atendimento as 0
Cliente 1 inicia atendimento as 0
Cliente 2 solicita atendimento as 2
Cliente 2 inicia atendimento as 2
Cliente 3 solicita atendimento as 11
Cliente 3 inicia atendimento as 11
Cliente 4 solicita atendimento as 21
Cliente 4 inicia atendimento as 21
Cliente 5 solicita atendimento as 31
Cliente 6 solicita atendimento as 35
Cliente 7 solicita atendimento as 42
Cliente 8 solicita atendimento as 52
Cliente 9 solicita atendimento as 56
Cliente 10 solicita atendimento as 61
Cliente 11 solicita atendimento as 66
Cliente 12 solicita atendimento as 72
Cliente 13 solicita atendimento as 82
Cliente 14 solicita atendimento as 87
Cliente 15 solicita atendimento as 90
Cliente 16 solicita atendimento as 94
Cliente 17 solicita atendimento as 101
Cliente 18 solicita atendimento as 106
Cliente 19 solicita atendimento as 111
Cliente 20 solicita atendimento as 117
Cliente 21 solicita atendimento as 124
Cliente 22 solicita atendimento as 126
Cliente 23 solicita aten

## Simulação com Melhoria